# Instructions

1.  Download this dataset:  https://www.kaggle.com/slothkong/10-monkey-species  
        You may need to create a kaggle account / sign in to accomplish this.
        
2. Git clone this repo or download this notebook.
3. Create a new folder, data in the same directory this notebook is located
4. Unzip the monkey dataset in this folder.  Then unzip the train.zip and validation.zip folders.
5. The directory hierarchy should be ../data/10-monkey-species/training and ../data/10-monkey-species/validation.  Each of these subdirectories should contain a directory per label, n0 thru n9.

6. Implement the code in build_model()

** You will most certainly need access to a GPU for this assignment **

## Questions

In addition to implementing build_model(), answer these questions.

1.  Is this network a high bias or high variance model?  Refer to chapter 1 and look at this model in tensorboard if you're unsure.

2. What is the likely cause of the issue you identified in the previous question?

3.  What might you do to make this network perform better?

In [ ]:
# these seeds are both required for reproducibility
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import os

In [ ]:
class Configuration:
    def __init__(self):
        self.feature_extraction_epochs = 10
        self.fine_tuning_epochs = 20
        self.epochs_without_transfer_learning = 100
        self.batch_size = 30
        self.data_dir = "data/10-monkey-species/training"
        self.val_dir = "data/10-monkey-species/validation"

In [ ]:
def build_model_feature_extraction():
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # make sure we're back at a 2d tensor
    x = Dense(1024, activation='relu')(x)  # add one fully connected layer
    predictions = Dense(10, activation='softmax')(x)  # 10 monkey, therefore 10 output units w softmax activation

    model = Model(inputs=base_model.input, outputs=predictions)

    # make the base model untrainable (frozen)
    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model




In [ ]:
def build_model_fine_tuning(model, learning_rate=0.0001, momentum=0.9):

        for layer in model.layers[:249]:
            layer.trainable = False
        for layer in model.layers[249:]:
            layer.trainable = True
        model.compile(optimizer=SGD(lr=learning_rate, momentum=momentum), loss='categorical_crossentropy', metrics=['accuracy'])
        return model

In [ ]:
def create_callbacks(name):
    tensorboard_callback = TensorBoard(log_dir=os.path.join(os.getcwd(), "tensorboard_log", name), write_graph=True, write_grads=False)
    checkpoint_callback = ModelCheckpoint(filepath="./model-weights-" + name + ".{epoch:02d}-{val_loss:.6f}.hdf5", monitor='val_loss',
                                          verbose=0, save_best_only=True)
    return [tensorboard_callback]

In [ ]:
def setup_data(train_data_dir, val_data_dir, img_width=299, img_height=299, batch_size=16):
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    return train_generator, validation_generator

In [ ]:
def fit_model(model, train_generator, val_generator, batch_size, epochs, name):
    model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.n // batch_size,
        callbacks=create_callbacks(name=name),
        verbose=1)
    return model

In [ ]:
def eval_model(model, val_generator, batch_size):
    scores = model.evaluate_generator(val_generator, steps=val_generator.n // batch_size)
    print("Loss: " + str(scores[0]) + " Accuracy: " + str(scores[1]))

## Load configuration class
This has all our model configuration parameters in it.  It's defined above.

In [ ]:
config = Configuration()

## Load Data
If everything is setup correctly, this method will print  
"  
Found 1097 images belonging to 10 classes.  
Found 272 images belonging to 10 classes.  
"  



In [ ]:
train_generator, val_generator = setup_data(config.data_dir, config.val_dir, batch_size=config.batch_size)

## Feature Extraction Model Training

Here we will train the feature extraction model

In [ ]:
model = build_model_feature_extraction()

Homework Q1: How many trainable parameters does this model have? How many more parameters exist in this model than the model we built last week?
    
Homework Q2: How can we possibly train a network of this size on a small dataset without overfitting?

In [ ]:
model = fit_model(model, train_generator, val_generator,
                  batch_size=config.batch_size,
                  epochs=config.feature_extraction_epochs,
                  name="feature_extraction")

In [ ]:
eval_model(model, val_generator, batch_size=config.batch_size)

## Fine Tuning 

Here we will fine tune that same model

In [ ]:
model = build_model_fine_tuning(model)
model = fit_model(model, train_generator, val_generator,
                      batch_size=config.batch_size,
                      epochs=config.fine_tuning_epochs,
                      name="fine_tuning")

Homework Q3: Did fine tuning help significantly?
    
Homework Q4: Imagine we wanted to use the InceptionV3 model to create a classifier for detecting skin cancer.  You have 200 images of both cancerous and benign abnormal skin cells.  Do you expect Fine Tuning to be useful in solving this problem, or will feature selection be sufficient?

In [ ]:
eval_model(model, val_generator, batch_size=config.batch_size)

In [ ]:
print("Saving Model...")
model.save("transfer_learning_model.h5")
